In [ ]:
import ee
import geemap
import ipywidgets as widgets
import ipyleaflet
from ipyleaflet import LayersControl
from IPython.display import display, FileLink, HTML
import requests
import os
import re

# ---------------- Initialize EE ----------------
def initialize_ee():
    try:
        ee.Initialize()
    except Exception:
        ee.Authenticate()
        ee.Initialize()

initialize_ee()

# ---------------- Constants ----------------
STATE_FC  = ee.FeatureCollection("TIGER/2018/States")
COUNTY_FC = ee.FeatureCollection("TIGER/2018/Counties")

_state_info = STATE_FC.getInfo()['features']
STATE_NAMES = sorted(f['properties']['NAME'] for f in _state_info)
STATE_FP_MAP = {f['properties']['NAME']: f['properties']['STATEFP'] for f in _state_info}

thermal_vis = {'min': -10, 'max': 40, 'palette': ['blue', 'green', 'yellow', 'red']}
ndvi_vis    = {'min': 0,   'max': 1,  'palette': ['white', 'lightgreen', 'darkgreen']}
ndbi_vis    = {'min': -1,  'max': 1,  'palette': ['brown', 'white', 'blue']}
utfvi_vis   = {'min': -0.5,'max': 0.5,'palette': ['green', 'white', 'red']}
hotspot_vis = {'min': 0,   'max': 1,  'palette': ['lightgray', 'red']}

# ---------------- Data Functions ----------------
def get_landsat_params(year, param):
    if year <= 2011:
        coll = 'LANDSAT/LT05/C02/T1_L2'
        bands = {'LST': 'ST_B6', 'NDVI': ('SR_B4','SR_B3'), 'NDBI': ('SR_B5','SR_B4')}
    elif year <= 2013:
        coll = 'LANDSAT/LE07/C02/T1_L2'
        bands = {'LST': 'ST_B6', 'NDVI': ('SR_B4','SR_B3'), 'NDBI': ('SR_B5','SR_B4')}
    elif year <= 2022:
        coll = 'LANDSAT/LC08/C02/T1_L2'
        bands = {'LST': 'ST_B10', 'NDVI': ('SR_B5','SR_B4'), 'NDBI': ('SR_B6','SR_B5')}
    else:
        coll = 'LANDSAT/LC09/C02/T1_L2'
        bands = {'LST': 'ST_B10', 'NDVI': ('SR_B5','SR_B4'), 'NDBI': ('SR_B6','SR_B5')}
    return coll, bands[param]

def apply_mask(img, use_radsat=False):
    qa = img.select('QA_PIXEL').bitwiseAnd(0b11000).eq(0)
    img = img.updateMask(qa)
    if use_radsat and img.bandNames().contains('QA_RADSAT'):
        img = img.updateMask(img.select('QA_RADSAT').eq(0))
    return img

def get_annual_lst(year, region):
    coll, band = get_landsat_params(year, 'LST')
    def toC(image):
        return image.select(band).multiply(0.00341802).add(149).subtract(273.15).rename('surface_temp')
    return ee.ImageCollection(coll)\
            .filterBounds(region).filterDate(f'{year}-01-01', f'{year}-12-31')\
            .map(lambda i: apply_mask(i, True))\
            .map(toC).median().clip(region).selfMask()

def get_ndvi(year, region):
    coll, (nir, red) = get_landsat_params(year, 'NDVI')
    return ee.ImageCollection(coll)\
            .filterBounds(region).filterDate(f'{year}-01-01', f'{year}-12-31')\
            .map(lambda i: apply_mask(i))\
            .map(lambda i: i.normalizedDifference([nir, red]).rename('NDVI'))\
            .median().clip(region)

def get_ndbi(year, region):
    coll, (swir, nir) = get_landsat_params(year, 'NDBI')
    return ee.ImageCollection(coll)\
            .filterBounds(region).filterDate(f'{year}-01-01', f'{year}-12-31')\
            .map(lambda i: apply_mask(i))\
            .map(lambda i: i.normalizedDifference([swir, nir]).rename('NDBI'))\
            .median().clip(region)

# ---------------- Map Interaction ----------------
def update_counties(change):
    state = change['new']
    fips = STATE_FP_MAP[state]
    names = COUNTY_FC.filter(ee.Filter.eq('STATEFP', fips))\
                     .aggregate_array('NAME').distinct().sort().getInfo()
    county_dd.options = names or ['(none)']
    county_dd.value = names[0] if names else '(none)'

def add_layer(map_obj, image, vis, name):
    try:
        return map_obj.add_layer(image, vis, name)
    except:
        return map_obj.addLayer(image, vis, name)

_prev_layer = None
REGION = None

def update_map(*_):
    global _prev_layer, REGION
    if _prev_layer:
        try:
            m.remove_layer(_prev_layer)
        except:
            m.removeLayer(_prev_layer)

    region_type = region_type_dd.value
    state = state_dd.value
    yr = int(year_dd.value)
    p = param_dd.value

    if region_type == 'State':
        feat = STATE_FC.filter(ee.Filter.eq('NAME', state)).first()
    else:
        fips = STATE_FP_MAP[state]
        feat = COUNTY_FC.filter(ee.Filter.eq('STATEFP', fips))\
                        .filter(ee.Filter.eq('NAME', county_dd.value)).first()

    REGION = feat.geometry()

    if p == 'LST':
        img, vis = get_annual_lst(yr, REGION), thermal_vis
    elif p == 'NDVI':
        img, vis = get_ndvi(yr, REGION), ndvi_vis
    elif p == 'NDBI':
        img, vis = get_ndbi(yr, REGION), ndbi_vis
    elif p == 'UTFVI':
        lst = get_annual_lst(yr, REGION)
        mv = lst.reduceRegion(ee.Reducer.mean(), REGION, scale_dd.value).get('surface_temp')
        img = lst.subtract(ee.Number(mv)).divide(ee.Number(mv)).rename('UTFVI')
        vis = utfvi_vis
    else:
        lst = get_annual_lst(yr, REGION)
        stats = lst.reduceRegion(ee.Reducer.mean().combine(ee.Reducer.stdDev(), None, True), REGION, scale_dd.value)
        mv = stats.get('surface_temp_mean')
        sd = stats.get('surface_temp_stdDev')
        img = lst.gt(ee.Number(mv).add(ee.Number(sd).multiply(2))).rename('Hotspot')
        vis = hotspot_vis

    img = img.clip(REGION)
    _prev_layer = add_layer(m, img, vis, f"{p} {yr}")
    m.centerObject(feat, 8)
    info_lbl.value = f"Displayed {p} {yr} for {region_type}"

# ---------------- Download Functions ----------------
def get_safe_region_name():
    if region_type_dd.value == 'County':
        region_name = county_dd.value
    else:
        region_name = state_dd.value
    return re.sub(r'\W+', '_', region_name)

def download_map(b):
    if REGION is None:
        info_lbl.value = "Select a region first."
        return
    yr, p = int(year_dd.value), param_dd.value
    img = get_annual_lst(yr, REGION) if p == 'LST' else get_ndvi(yr, REGION)
    vis = thermal_vis if p == 'LST' else ndvi_vis

    url = img.getThumbURL({
        'min': vis['min'],
        'max': vis['max'],
        'palette': vis['palette'],
        'region': REGION,
        'dimensions': int(resolution_dd.value),
        'format': 'png'
    })
    r = requests.get(url)
    fn = f"map_{p}_{get_safe_region_name()}_{yr}.png"
    with open(fn, 'wb') as f:
        f.write(r.content)
    info_lbl.value = f"Saved {fn}"
    display(FileLink(fn))

def draw_year_label(year, region):
    point = ee.Geometry.Point(region.centroid().coordinates())
    return ee.Image().paint(point, year).visualize(min=0, max=2050, palette=['black'])

def download_anim(b):
    if REGION is None:
        info_lbl.value = "Select a region first."
        return

    start, end = int(anim_start_dd.value), int(anim_end_dd.value)
    yrs = range(start, end + 1, int(step_dd.value) or 1)
    frames = []

    for yr in yrs:
        if param_dd.value == 'LST':
            img = get_annual_lst(yr, REGION).visualize(**thermal_vis)
        elif param_dd.value == 'NDVI':
            img = get_ndvi(yr, REGION).visualize(**ndvi_vis)
        elif param_dd.value == 'NDBI':
            img = get_ndbi(yr, REGION).visualize(**ndbi_vis)
        elif param_dd.value == 'UTFVI':
            lst = get_annual_lst(yr, REGION)
            mv = lst.reduceRegion(ee.Reducer.mean(), REGION, scale_dd.value).get('surface_temp')
            img = lst.subtract(ee.Number(mv)).divide(ee.Number(mv)).rename('UTFVI').visualize(**utfvi_vis)
        else:
            lst = get_annual_lst(yr, REGION)
            stats = lst.reduceRegion(ee.Reducer.mean().combine(ee.Reducer.stdDev(), None, True), REGION, scale_dd.value)
            mv = stats.get('surface_temp_mean')
            sd = stats.get('surface_temp_stdDev')
            img = lst.gt(ee.Number(mv).add(ee.Number(sd).multiply(2))).rename('Hotspot').visualize(**hotspot_vis)

        label = draw_year_label(yr, REGION)
        img = ee.ImageCollection([img, label]).mosaic()
        frames.append(img)

    col = ee.ImageCollection(frames)
    url = col.getVideoThumbURL({
        'region': REGION,
        'dimensions': int(resolution_dd.value),
        'framesPerSecond': 1,
        'format': 'gif',
        'backgroundColor': 'white'
    })
    r = requests.get(url)
    fn = f"anim_{param_dd.value}_{get_safe_region_name()}_{start}_{end}.gif"
    with open(fn, 'wb') as f:
        f.write(r.content)

    info_lbl.value = f"Saved {fn}"
    display(HTML(f'<img src="{fn}" style="max-width:100%;">'))

# ---------------- UI ----------------
year_dd = widgets.Dropdown(options=list(range(1985, 2025)), value=2024, description='Year:')
param_dd = widgets.Dropdown(options=['LST','NDVI','NDBI','UTFVI','Hotspots'], value='LST', description='Param:')
state_dd = widgets.Dropdown(options=STATE_NAMES, value='Florida', description='State:')
region_type_dd = widgets.Dropdown(options=['State','County'], value='State', description='Type:')
county_dd = widgets.Dropdown(options=[], description='County:')
resolution_dd = widgets.IntText(value=256, description='Res(px):')
scale_dd = widgets.IntText(value=1000, description='Scale(m):')
download_map_btn = widgets.Button(description='Download Map')
anim_start_dd = widgets.Dropdown(options=list(range(1985, 2025)), value=1985, description='Anim Start:')
anim_end_dd = widgets.Dropdown(options=list(range(1985, 2025)), value=2024, description='Anim End:')
step_dd = widgets.IntSlider(value=1, min=1, max=10, description='Step:')
animate_btn = widgets.Button(description='Download Animation')
info_lbl = widgets.Label()

state_dd.observe(update_counties, names='value')
for w in [param_dd, year_dd, region_type_dd, county_dd]:
    w.observe(update_map, names='value')
download_map_btn.on_click(download_map)
animate_btn.on_click(download_anim)
update_counties({'new': state_dd.value})

map_ctrl = widgets.HBox([param_dd, year_dd, state_dd, region_type_dd, county_dd, resolution_dd, download_map_btn])
anim_ctrl = widgets.HBox([step_dd, anim_start_dd, anim_end_dd, animate_btn])
ui = widgets.VBox([map_ctrl, anim_ctrl, info_lbl])

m = geemap.Map()
styled = COUNTY_FC.style(color='0000FF', fillColor='00000000', width=1)
add_layer(m, styled, {}, 'County Boundaries')
m.add_control(LayersControl(position='topright'))

display(ui, m)

_prev_layer = None
update_map()
